In [ ]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.ml import Pipeline
from pyspark.ml import PipelineModel

# Create a local StreamingContext with two working thread and batch interval of 1 second
sc = SparkContext(appName="LogParser-py-Streaming Mod11")
ssc = StreamingContext(sc, 120)

In [ ]:
from pyspark.ml import PipelineModel

In [ ]:
from pyspark.sql import SQLContext
from pyspark.sql import Row
import sys
from datetime import datetime

In [ ]:
#Load the model
#model = PipelineModel.load('use_cases/spam_model')

In [ ]:
now = datetime.now()
print(now)

In [ ]:
filepath = '/user/edureka_524533/Flume_Spam/' + now.strftime('%Y-%m-%d') 
print(filepath)

In [ ]:
def getSparkSessionInstance(sparkConf):
        if ("sparkSessionSingletonInstance" not in globals()):
            globals()["sparkSessionSingletonInstance"] = SparkSession \
                .builder \
                .config(conf=sparkConf) \
                .getOrCreate()
        return globals()["sparkSessionSingletonInstance"]

In [ ]:
# Create a DStream that will connect to hostname:port, like localhost:9999
#lines = ssc.socketTextStream("localhost", 9999)
lines = ssc.textFileStream(filepath)

In [ ]:
def process(t, rdd):
    if rdd.isEmpty():
            print("==== EMPTY ====")
            return
    print("=== RDD Found ===")
    rowRdd = rdd.map(lambda x: Row(line=x))
    spark = getSparkSessionInstance(rdd.context.getConf())
    df = spark.createDataFrame(rowRdd)
    df.printSchema()
    df = df.withColumnRenamed('line','message')
    df.printSchema()
    pipeline = PipelineModel.load("use_cases/spam_model4.4")
    predictions = pipeline.transform(df)
    predictions.select('features','message','prediction').show()
    

In [ ]:
lines.pprint()
lines.foreachRDD(process)

In [ ]:
ssc.start()             # Start the computation
ssc.awaitTermination()  # Wait for the computation to terminate